## Encoding Categorical Variables Using Statistics
[Source](https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/discussion/32123)

In [25]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train = pd.read_json('train.json')
test = pd.read_json('test.json')
print(train.shape)
print(test.shape)

(49352, 15)
(74659, 14)


In [26]:
train['row_id'] = range(train.shape[0])
test['row_id'] = range(test.shape[0])
train['train'] = 1
test['train'] = 0

In [27]:
target_column = 'price'
group_column = 'manager_id'

data = train[['row_id', 'train', target_column, group_column]].append(
    test[['row_id','train', target_column, group_column]])
data.head(2)

,row_id,train,price,manager_id
10,0,1,3000,5ba989232d0489da1b5f2c45f6688adc
10000,1,1,5465,7533621a882f71e25173b27e3139d83d


In [28]:
grouped = data[[target_column, group_column]].groupby(group_column)
grouped

In [29]:
len(data.price[data.manager_id=='0000abd7518b94c35a90d64b56fbf3e6'])

35

In [30]:
the_size = pd.DataFrame(grouped.size()).reset_index()
the_size.columns = [group_column, '%s_size' % target_column]
the_size.head(2)

,manager_id,price_size
0,0000abd7518b94c35a90d64b56fbf3e6,35
1,001ce808ce1720e24a9510e014c69707,28


In [31]:
sum(data.price[data.manager_id=='0000abd7518b94c35a90d64b56fbf3e6']) / \
len(data.price[data.manager_id=='0000abd7518b94c35a90d64b56fbf3e6'])

2765.1714285714284

In [32]:
the_mean = pd.DataFrame(grouped.mean()).reset_index()
the_mean.columns = [group_column, '%s_mean' % target_column]
the_mean.head(2)

,manager_id,price_mean
0,0000abd7518b94c35a90d64b56fbf3e6,2765.171429
1,001ce808ce1720e24a9510e014c69707,3170.535714


In [33]:
the_std = pd.DataFrame(grouped.std()).reset_index().fillna(0)
the_std.columns = [group_column, '%s_std' % target_column]
the_median = pd.DataFrame(grouped.median()).reset_index()
the_median.columns = [group_column, '%s_median' % target_column]
the_max = pd.DataFrame(grouped.max()).reset_index()
the_max.columns = [group_column, '%s_max' % target_column]
the_min = pd.DataFrame(grouped.min()).reset_index()
the_min.columns = [group_column, '%s_min' % target_column]

In [34]:
the_stats = pd.merge(the_size, the_mean).merge(the_std).merge(the_median).merge(the_max).merge(the_min)
the_stats.head(2)

,manager_id,price_size,price_mean,price_std,price_median,price_max,price_min
0,0000abd7518b94c35a90d64b56fbf3e6,35,2765.171429,532.174976,2695.0,4295,1995
1,001ce808ce1720e24a9510e014c69707,28,3170.535714,1116.447192,3000.0,6495,1795


In [35]:
print(the_stats.shape)
print(data.shape)

(4399, 7)
(124011, 4)


In [36]:
data = pd.merge(data, the_stats)
data.head()

,row_id,train,price,manager_id,price_size,price_mean,price_std,price_median,price_max,price_min
0,0,1,3000,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700
1,107,1,2250,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700
2,152,1,2200,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700
3,572,1,1950,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700
4,1198,1,2000,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700


In [37]:
selected_train = data[data['train'] == 1]
selected_test = data[data['train'] == 0]
print(selected_train.shape)
print(selected_test.shape)

(49352, 10)
(74659, 10)


In [38]:
selected_train.sort_values('row_id', inplace=True)
selected_test.sort_values('row_id', inplace=True)

In [39]:
selected_train.head()

,row_id,train,price,manager_id,price_size,price_mean,price_std,price_median,price_max,price_min
0,0,1,3000,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700
235,1,1,5465,7533621a882f71e25173b27e3139d83d,194,5419.932990,1905.188512,4700.0,12995,1995
429,2,1,2850,d9039c43983f6e564b1482b273bd7b01,314,3618.716561,1318.133209,3295.0,8795,1650
743,3,1,3275,1067e078446a7897d2da493d2f741316,470,3701.768085,1392.090938,3450.0,11100,1650
1213,4,1,3350,98e13ad4b495b9613cef886d79a6291f,51,3114.901961,1328.458765,2870.0,7995,1495


In [40]:
train[['manager_id', 'interest_level']].head()

,manager_id,interest_level
10,5ba989232d0489da1b5f2c45f6688adc,medium
10000,7533621a882f71e25173b27e3139d83d,low
100004,d9039c43983f6e564b1482b273bd7b01,high
100007,1067e078446a7897d2da493d2f741316,low
100013,98e13ad4b495b9613cef886d79a6291f,low


In [17]:
selected_train.drop([target_column, group_column, 'row_id', 'train'], axis=1, inplace=True)
selected_test.drop([target_column, group_column, 'row_id', 'train'], axis=1, inplace=True)

In [18]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
selected_train.reset_index(drop=True, inplace=True)
selected_test.reset_index(drop=True, inplace=True)

In [19]:
selected_train.head()

,price_size,price_mean,price_std,price_median,price_max,price_min
0,235,2541.914894,524.703572,2600.0,4400,1700
1,194,5419.932990,1905.188512,4700.0,12995,1995
2,314,3618.716561,1318.133209,3295.0,8795,1650
3,470,3701.768085,1392.090938,3450.0,11100,1650
4,51,3114.901961,1328.458765,2870.0,7995,1495


In [20]:
selected_train.shape

(49352, 6)

In [21]:
selected_test.shape

(74659, 6)

In [24]:
train.head(2)

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,row_id,train
0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,0,1
1,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,1,1


In [22]:
X_train = pd.merge(train, selected_train, left_index=True, right_index=True)
X_train.shape

(49352, 23)

In [ ]:
#Identify top features using a basic XGBoost
# I'd like to thank this from 
#https://www.kaggle.com/sudalairajkumar/sberbank-russian-housing-market/simple-exploration-notebook-sberbank
#It helped me to understand a simple way to build my feature importance
for f in train_data:
    if train_data[f].dtype == "object":
        lbl=pre.LabelEncoder()
        lbl.fit(list(train_data[f].values))
        train_data[f]=lbl.transform(list(train_data[f].values))

xgb_params = {
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

y_train = train_data["price_doc"]
x_train = train_data.drop(["id","timestamp","price_doc"],axis = 1)
dtrain = xgb.DMatrix(x_train,y_train,feature_names = x_train.columns.values)
model = xgb.train(dict(xgb_params,silent=0),dtrain,num_boost_round=100)

fig,ax=plt.subplots(figsize = (12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [9]:
def get_stats(train, test, target_column, group_column='manager_id'):
    '''
    target_column: numeric columns to group with (e.g. price, bedrooms, bathrooms)
    group_column: categorical columns to group on (e.g. manager_id, building_id)
    '''
    train['row_id'] = range(train.shape[0])
    test['row_id'] = range(test.shape[0])
    train['train'] = 1
    test['train'] = 0
    data = train[['row_id', 'train', target_column, group_column]].append(
        test[['row_id','train', target_column, group_column]])
    
    grouped = data[[target_column, group_column]].groupby(group_column)
    the_size = pd.DataFrame(grouped.size()).reset_index()
    the_size.columns = [group_column, '%s_size' % target_column]
    the_mean = pd.DataFrame(grouped.mean()).reset_index()
    the_mean.columns = [group_column, '%s_mean' % target_column]
    the_std = pd.DataFrame(grouped.std()).reset_index().fillna(0)
    the_std.columns = [group_column, '%s_std' % target_column]
    the_median = pd.DataFrame(grouped.median()).reset_index()
    the_median.columns = [group_column, '%s_median' % target_column]
    the_max = pd.DataFrame(grouped.max()).reset_index()
    the_max.columns = [group_column, '%s_max' % target_column]
    the_min = pd.DataFrame(grouped.min()).reset_index()
    the_min.columns = [group_column, '%s_min' % target_column]
    
    the_stats = pd.merge(
        the_size, the_mean).merge(the_std).merge(the_median).merge(the_max).merge(the_min)

    data = pd.merge(data, the_stats)

    selected_train = data[data['train'] == 1]
    selected_test = data[data['train'] == 0]
    selected_train.sort_values('row_id', inplace=True)
    selected_test.sort_values('row_id', inplace=True)
    selected_train.drop([target_column, 'row_id', 'train'], axis=1, inplace=True)
    selected_test.drop([target_column, 'row_id', 'train'], axis=1, inplace=True)

    return selected_train, selected_test

In [10]:
selected_train, selected_test = get_stats(train, test, 'price', group_column='manager_id')

In [18]:
selected_train.head()

,manager_id,price_size,price_mean,price_std,price_median,price_max,price_min
0,5ba989232d0489da1b5f2c45f6688adc,235,2541.914894,524.703572,2600.0,4400,1700
235,7533621a882f71e25173b27e3139d83d,194,5419.932990,1905.188512,4700.0,12995,1995
429,d9039c43983f6e564b1482b273bd7b01,314,3618.716561,1318.133209,3295.0,8795,1650
743,1067e078446a7897d2da493d2f741316,470,3701.768085,1392.090938,3450.0,11100,1650
1213,98e13ad4b495b9613cef886d79a6291f,51,3114.901961,1328.458765,2870.0,7995,1495


In [19]:
train[['manager_id', 'interest_level']].head()

,manager_id,interest_level
10,5ba989232d0489da1b5f2c45f6688adc,medium
10000,7533621a882f71e25173b27e3139d83d,low
100004,d9039c43983f6e564b1482b273bd7b01,high
100007,1067e078446a7897d2da493d2f741316,low
100013,98e13ad4b495b9613cef886d79a6291f,low


In [59]:
selected_manager_id_proj = ['bathrooms', 'bedrooms', 'listing_id']

for target_col in selected_manager_id_proj:
    tmp_train, tmp_test = get_stats(train_df, test_df, target_column=target_col)
    train_stack = pd.merge(train_stack, tmp_train)
#     train_stack_list.append(tmp_train)
#     test_stack_list.append(tmp_test)

In [60]:
train_stack.head(2)

,row_id,bathrooms_size,bathrooms_mean,bathrooms_std,bathrooms_median,bathrooms_max,bathrooms_min,bedrooms_size,bedrooms_mean,bedrooms_std,bedrooms_median,bedrooms_max,bedrooms_min,listing_id_size,listing_id_mean,listing_id_std,listing_id_median,listing_id_max,listing_id_min,manager_id
0,0,235,1.106383,0.305508,1.0,2.0,1.0,235,1.502128,0.935412,1.0,4,0,235,7031766,125976.544428,7050454,7226750,6814711,5ba989232d0489da1b5f2c45f6688adc
1,1,194,1.360825,0.544603,1.0,3.5,0.0,194,1.623711,0.831787,1.0,5,0,194,7120468,74552.553594,7146956,7231089,7011029,7533621a882f71e25173b27e3139d83d


In [67]:
train_df_index = train_df.reset_index()
train_df_index.head(1)

,index,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,row_id,train
0,10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,0,1


In [74]:
train_stack.manager_id.tail(2)

49350    9fd3af5b2d23951e028059e8940a55d7
49351    d7f57128272bfd82e33a61999b5f4c42
Name: manager_id, dtype: object

In [73]:
train_df_index.manager_id.tail(2)

49350    9fd3af5b2d23951e028059e8940a55d7
49351    d7f57128272bfd82e33a61999b5f4c42
Name: manager_id, dtype: object

In [77]:
sum(train_stack.manager_id.values == train_df.manager_id.values)

49352

In [ ]:
selected_manager_id_proj = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price', 'bad_addr', 'listing_id',
                   'month', 'day', 'weekday', 'day_of_year', 'hour', 'num_features', 'num_desc',
                   'bed_to_bath', 'price_per_bed', 'price_per_bath', 'bldg_count', 'zero_bldg', 'total_room', 'room_diff',
                   'photo_count', 'latitude_grid', 'longitude_grid', 'lat_long_grid']

for target_col in selected_manager_id_proj:
    tmp_train, tmp_test = get_stats(train_df, test_df, target_column=target_col)
    train_stack_list.append(tmp_train)
    test_stack_list.append(tmp_test)

selected_bedrooms_proj = ['price', 'listing_id', 'month', 'day', 'weekday', 'day_of_year', 'hour', 'num_features', 'bldg_count', 'zero_bldg']

for target_col in selected_bedrooms_proj:
    tmp_train, tmp_test = get_stats(train_df, test_df, target_column=target_col, group_column='bedrooms')
    train_stack_list.append(tmp_train)
    test_stack_list.append(tmp_test)